In [2]:
from pyspark.sql import SparkSession
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point

In [3]:
# Create a new Spark session (if not already created)
spark = SparkSession.builder.appName("Data Visualization").getOrCreate()

# Read the filtered data from the saved Parquet file
Crash_results_filtered = spark.read.parquet('../data/landing/Crash_results_filtered.parquet')

your 131072x1 screen size is bogus. expect trouble
23/08/01 14:21:54 WARN Utils: Your hostname, LAPTOP-RELH58H1 resolves to a loopback address: 127.0.1.1; using 172.19.22.4 instead (on interface eth0)
23/08/01 14:21:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/01 14:21:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Sample 5% of the data
sampled_data = Crash_results_filtered.sample(fraction=0.000005, seed=42)

# Convert Spark DataFrame to Pandas DataFrame
crash_results_pd = sampled_data.toPandas()

# Create a GeoPandas DataFrame with the spatial information (latitude and longitude)
geometry = [Point(xy) for xy in zip(crash_results_pd["LONGITUDE"], crash_results_pd["LATITUDE"])]
crash_results_geo = gpd.GeoDataFrame(crash_results_pd, crs="EPSG:4326", geometry=geometry)

# Initialize the map with a center location and zoom level
_map = folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)

# Create a GeoJSON data from the GeoDataFrame (crash_results_geo) to use with Folium
geoJSON = crash_results_geo.__geo_interface__

# Add the choropleth layer to the map
_map.add_child(folium.Choropleth(
    geo_data=geoJSON,
    name='choropleth',
))

# Save the map as an HTML file
_map.save('../plots/foliumChoroplethMap.html')

# Display the map
_map


23/08/01 14:22:09 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
